In [10]:
!pip install tifffile
!pip install --upgrade tensorflow


import glob
import sys
import random
import warnings

import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

import tifffile as tiff


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import tensorflow as tf
# gpus = tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    except RuntimeError as e:
        print(e)

In [12]:
from keras import backend as K

color_dict = {
    # 0: [  0.,   0., 0.],
    # 1: [255., 255., 255.]
           # 0 : (0,0,0),  #no data
             0: [0.,0.,0.],
           # 1 : (128, 255, 0), #forest -
             1 : [128., 255., 0.],
           # 2 : (166, 0, 204), #mine-
             2 : [166., 0., 204.],
           # 3 : (204, 0, 0), #road-
             3 : [204., 0., 0.],
           # 4 : (255, 0, 0), #disc urban   -
             4 : [255., 0., 0.],
           # 5 : (255, 255, 168), #arable land-
             5 : [255., 255., 168.],
           # 6 : (255,230,166), #heteregonous agricultural areas -
             6 : [255.,230.,166.],
           # 7 : (231, 40, 252), #permenant crops -
             7 : [231., 40., 252.],
           # 8 : (0, 204, 242),  #inland waters -
             8 : [0., 204., 242.],
           # 9 : (255, 166, 255),  #artificial non-agricultural vegetated areas-
            9 : [255., 166., 255.],
}

def rgb_to_onehot(rgb_arr,color_dict=color_dict):
    num_classes = len(color_dict)
    shape = rgb_arr.shape[:2]+(num_classes,)
    arr = np.zeros(shape,dtype = np.int8)
    for i,cls in enumerate(color_dict):
        arr[:,:,i] = np.all(rgb_arr.reshape((-1,3))==color_dict[i],axis=1).reshape(shape[:2])
    arr = np.array(arr)
    mask = np.argmax(arr,axis=-1)
    return mask

def onehot_to_rgb(onehot,color_dict = color_dict):
    output = np.zeros(onehot.shape[:2]+(3,))
    for k in color_dict.keys():
        output[onehot==k]=color_dict[k]
    return output

def adjustData(img, mask):
  onehot=rgb_to_onehot(mask,color_dict)
  onehot=np.reshape(onehot,((1,)+onehot.shape))
  return(img, onehot)

def TrainGenerator(aug_dict, image_folder, mask_folder, batch_size, save_to_dir, seed = 1, image1_save_prefix = "image",  mask_save_prefix = "mask"):
  image_datagen = ImageDataGenerator(**aug_dict)
  mask_datagen = ImageDataGenerator(**aug_dict)

  image_generator = image_datagen.flow_from_directory(
       directory = image_folder,
       target_size = (512, 512),
       color_mode = 'rgb',
       class_mode = None,
       batch_size = batch_size,
       save_to_dir = save_to_dir,
       save_prefix = image1_save_prefix,
       shuffle = True,
       seed = seed
  )
  mask_generator = mask_datagen.flow_from_directory(
      directory = mask_folder,
      target_size = (512, 512),
       color_mode = 'rgb',
       class_mode = None,
       batch_size = batch_size,
       save_to_dir = save_to_dir,
       save_prefix = mask_save_prefix,
       shuffle = True,
       seed = seed
  )
  train_generator = zip(image_generator, mask_generator)
  for (img, mask) in train_generator:
        X=[]
        Y=[]
        for d in range(img.shape[0]):
            x, y = adjustData(img[d], mask[d])
            X.append(x)
            Y.append(y)
        X = np.reshape(np.array(X),(img.shape[0],512, 512,3))
        Y = np.reshape(np.array(Y),(img.shape[0],512, 512,1))
        yield X, Y

def ValGenerator(aug_dict, image_folder, mask_folder, batch_size, save_to_dir, seed = 1, image_save_prefix = "image", mask_save_prefix = "mask"):
  image_datagen = ImageDataGenerator(**aug_dict)
  mask_datagen = ImageDataGenerator(**aug_dict)

  image_generator = image_datagen.flow_from_directory(
       directory = image_folder,
       target_size = (512, 512),
       color_mode = 'rgb',
       class_mode = None,
       batch_size = batch_size,
       save_to_dir = save_to_dir,
       save_prefix = image_save_prefix,
       shuffle = True,
       seed = seed
  )
  mask_generator = mask_datagen.flow_from_directory(
      directory = mask_folder,
      target_size = (512, 512),
       color_mode = 'rgb',
       class_mode = None,
       batch_size = batch_size,
       save_to_dir = save_to_dir,
       save_prefix = mask_save_prefix,
       shuffle = True,
       seed = seed
  )
  train_generator = zip(image_generator, mask_generator)
  for (img, mask) in train_generator:
        X=[]
        Y=[]
        for d in range(img.shape[0]):
            x, y = adjustData(img[d], mask[d])
            X.append(x)
            Y.append(y)
        X = np.reshape(np.array(X),(img.shape[0],512, 512,3))
        Y = np.reshape(np.array(Y),(img.shape[0],512, 512,1))
        yield X, Y

In [13]:
import numpy as np
# from keras.backend import int_shape
from tensorflow.keras.backend import int_shape
from keras.models import Model
from keras.layers import Conv2D,Conv2DTranspose, Conv3D, MaxPooling2D, MaxPooling3D, UpSampling2D, Add, BatchNormalization, Input, Activation, Lambda, Concatenate
from tensorflow.keras import regularizers


def residual_separable(inp, n_filters, dropout=0.3, dilation=1, l2=None, name="down"):
    x = tf.keras.layers.SeparableConv2D(n_filters, (3, 3), strides=1, padding='same', activation=None,
                        dilation_rate=dilation, use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform',
                        pointwise_regularizer=regularizers.l2(0.00004))(inp)
    x = BatchNormalization()(x)
    x = Dropout(rate=dropout)(x)
    if inp.shape[3] == x.shape[3]:
        x = Add()([x, inp])
    x = Activation('relu')(x)
    return x

def residual_separable_multi(inp, n_filters, dropout=0.3, dilation=1, l2=None, name="down"):
    x = tf.keras.layers.DepthwiseConv2D(3, strides=(1, 1), depth_multiplier=1, padding='same', use_bias=False)(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x2 = tf.keras.layers.DepthwiseConv2D(3, strides=(1, 1), depth_multiplier=1, dilation_rate= (dilation, dilation), padding='same', use_bias=False)(inp)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)

    x +=x2

    x = Conv2D(n_filters, 1, strides=1, padding='same', activation=None, 
                         dilation_rate=1, use_bias=False, kernel_regularizer=regularizers.l2(0.00004))(x)
    x = BatchNormalization()(x)

    x = Dropout(rate=dropout)(x)

    if inp.shape[3] == x.shape[3]:
        x = Add()([x, inp])

    x = Activation('relu')(x)
    return x


def encoder_module(inp, n_filters, dropout=0.3, dilation=[1,1], l2=None, name="down"):
    x = residual_separable(inp, n_filters, dropout=dropout, dilation=dilation[0], l2=l2, name=name)
    x = residual_separable(x, n_filters, dropout=dropout, dilation=dilation[1], l2=l2, name=name)
    return x

def encoder_module_multi(inp, n_filters, dropout=0.3, dilation=[1,1], l2=None, name="down"):
    x = residual_separable_multi(inp, n_filters, dropout=dropout, dilation=dilation[0], l2=l2, name=name)
    x = residual_separable_multi(x, n_filters, dropout=dropout, dilation=dilation[1], l2=l2, name=name)
    return x

def upsample(x, n_filters, last=False, l2=None, name="down"):
    x = Conv2DTranspose(n_filters, 3, strides=2, padding='same',use_bias=True,
                                   activation=None,
                                   kernel_regularizer=regularizers.l2(0.00004))(x)
    if not last:
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    return x

def downsample(inp, n_filters_in, n_filters_out, bn=False, use_relu=False, l2=None, name="down"):
    
    if n_filters_in < n_filters_out:
        filters_conv = n_filters_out - n_filters_in
    else:
        filters_conv = n_filters_out

    x = Conv2D(filters_conv, 3, strides=2, padding='same', activation=None,
                         dilation_rate=1, use_bias=False, kernel_regularizer=regularizers.l2(0.00004))(inp)
    
    if n_filters_in < n_filters_out:
        y =  MaxPooling2D(pool_size=(2, 2), strides=(2,2))(inp)
        x =  concatenate([x,y], axis = -1)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def dpp(x,f):
  x1 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x2 = tf.keras.layers.SeparableConv2D(f, (3, 3), strides = 2, padding='same', activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x3 = tf.keras.layers.SeparableConv2D(f, (3, 3), strides = 4, padding='same', activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x1 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x1) 
  x2 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x2) 
  x3 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same', activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x3) 
  x1 = Conv2D(f,(1,1))(x1)
  x2 = Conv2DTranspose(f, 3, strides=2, padding='same',use_bias=True, activation='relu')(x2)
  x3 = Conv2DTranspose(f, 3, strides=4, padding='same',use_bias=True, activation='relu')(x3)  
  x = concatenate([x3,x2,x1], axis=3)
  x1 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (1,1), activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x2 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (2,2), activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x3 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (4,4), activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x4 = tf.keras.layers.SeparableConv2D(f, (3, 3), padding='same',dilation_rate = (8,8), activation='relu', 
                                       use_bias=False, depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform')(x) 
  x = concatenate([x1,x2,x3,x4], axis=3)
  x = Conv2D(f,(1,1))(x)
  return x
     

In [14]:
import numpy as np
# from keras.backend import int_shape
from tensorflow.keras.backend import int_shape
from keras.models import Model
from keras.layers import *
from tensorflow.keras import regularizers
import tensorflow as tf

def feature_extractor(x,f,dropout=0.25, dilation=[1,1,1,1], l2=None, name="down"):
  x1_1 = residual_separable_multi(x, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x1_2 = residual_separable_multi(x, f//4, dropout=dropout, dilation=dilation[0], l2=l2, name=name)
  x2 = concatenate([x,x1_2])
  x2_1 = residual_separable_multi(x2, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x2_2 = residual_separable_multi(x2_1, f//4, dropout=dropout, dilation=dilation[1], l2=l2, name=name)
  x3 = concatenate([x,x1_2,x2_2])
  x3_1 = residual_separable_multi(x3, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x3_2 = residual_separable_multi(x3_1, f//4, dropout=dropout, dilation=dilation[2], l2=l2, name=name)
  x4 = concatenate([x,x1_2,x2_2,x3_2])
  x4_1 = residual_separable_multi(x4, f, dropout=dropout, dilation=1, l2=l2, name=name)
  x4_2 = residual_separable_multi(x4_1, f//4, dropout=dropout, dilation=dilation[3], l2=l2, name=name)
  xout = concatenate([x,x1_2,x2_2,x3_2,x4_2])
  return xout

In [17]:
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D, DepthwiseConv2D, Conv2DTranspose
import tensorflow.keras.backend as K 
from keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Activation,Reshape, Add, Multiply, DepthwiseConv2D, BatchNormalization, Concatenate, Conv2D, Dense,Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Lambda,LeakyReLU, MaxPooling2D, Multiply, Permute, Reshape, UpSampling2D 
import collections
import tensorflow as tf
    

def dppnet(input_size,  n_classes, l2=None, upsampling=2):
    inputs = Input(input_size)
    d1 = downsample(inputs, 3, 16, l2=l2, name="d1")
    d2 = downsample(d1, n_filters_in=16, n_filters_out=64, l2=l2, name="d2")
    m1 = encoder_module(d2, n_filters=64, dilation=[1, 1], l2=l2, name="fres3", dropout=0.0)
    m2 = encoder_module(m1, n_filters=64, dilation=[1, 1], l2=l2, name="fres4", dropout=0.0)
    m3 = encoder_module(m2, n_filters=64, dilation=[1, 1], l2=l2, name="fres5", dropout=0.0)
    m4 = encoder_module(m3, n_filters=64, dilation=[1, 1], l2=l2, name="fres6", dropout=0.0)
    m5 = encoder_module(m4, n_filters=64, dilation=[1, 1], l2=l2, name="fres7", dropout=0.0)

    d3 = downsample(m5,  n_filters_in=64, n_filters_out=128, l2=l2, name="d8")
    m6 = feature_extractor(d3,128,dropout=0.25, dilation=[2,4,8,16], l2=l2, name="fres9")
    m7 = feature_extractor(m6,128,dropout=0.25, dilation=[1,2,8,16], l2=l2, name="fres10")
    up1 = upsample(m7, n_filters=64, l2=l2, name="up11")
    x = up1+d2
    up2 = upsample(x, n_filters=16, l2=l2, name="up16", last = True)
    x = concatenate([up2, d1], axis=3)
    x = dpp(x,32)
    x = Conv2D(1, (1, 1), activation='sigmoid')(x)
    if upsampling > 1:
        x = tf.keras.layers.Resizing(x.shape[1] * upsampling, x.shape[2] * upsampling)(x)

      # x = tf.keras.layers.experimental.preprocessing.Resizing(x.shape[1] * upsampling, x.shape[2] * upsampling)(x)


    model = Model(inputs, x)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = dppnet((512,512,3),8)
model.summary()

    


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_77 (Conv2D)  │ (None, 256, 256,  │        351 │ input_layer_4[0]… │
│                     │ 13)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_11    │ (None, 256, 256,  │          0 │ input_layer_4[0]… │
│ (MaxPooling2D)      │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_44      │ (None, 256, 256,  │          0 │ conv2d_77[0][0],  │
│ (Concatenate)       │ 16)               │            │ max_pooling2d_11… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │         64 │ concatenate_44[0… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_196      │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_78 (Conv2D)  │ (None, 128, 128,  │      6,912 │ activation_196[0… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 128, 128,  │          0 │ activation_196[0… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_45      │ (None, 128, 128,  │          0 │ conv2d_78[0][0],  │
│ (Concatenate)       │ 64)               │            │ max_pooling2d_12… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ concatenate_45[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_197      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_60 │ (None, 128, 128,  │      4,672 │ activation_197[0… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_78          │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_84 (Add)        │ (None, 128, 128,  │          0 │ dropout_78[0][0], │
│                     │ 64)               │            │ activation_197[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_198      │ (None, 128, 128,  │          0 │ add_84[0][0]      │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_61 │ (None, 128, 128,  │      4,672 │ activation_198[0

 Total params: 679,760 (2.59 MB)

 Trainable params: 665,136 (2.54 MB)

 Non-trainable params: 14,624 (57.12 KB)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen_args = dict()
Generator = TrainGenerator(aug_dict=data_gen_args,image_folder='/kaggle/input/dataset-kestal/Aksu/Train/Image',mask_folder='/kaggle/input/dataset-kestal/Aksu/Train/Mask',batch_size=2,save_to_dir=None)
# Generator = TrainGenerator(aug_dict=data_gen_args,image_folder='G:/Segmentation _Multisensor/building/512/Train/I/',mask_folder='G:/Segmentation _Multisensor/building/512/Train/G/',batch_size=2,save_to_dir=None)
# Val = ValGenerator(aug_dict=data_gen_args,image_folder='G:/Segmentation _Multisensor/building/512/Val/I/',mask_folder='G:/Segmentation _Multisensor/building/512/Val/G/',batch_size=2,save_to_dir=None)
Val = ValGenerator(aug_dict=data_gen_args,image_folder='/kaggle/input/dataset-kestal/Aksu/Validation/Image',mask_folder='/kaggle/input/dataset-kestal/Aksu/Validation/Mask',batch_size=2,save_to_dir=None)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',mode='min',factor=0.5,patience=10,verbose=1,cooldown=1,min_delta=0.0001)
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=20,verbose=1,mode='min',restore_best_weights=True)
# check_path = 'G:/Segmentation _Multisensor/Weights/Binary_Building/DPP_Massa_Building.h5'
# check_path = "/kaggle/working/model_checkpoint.weights.h5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=True,mode='min')
# history = model.fit(Generator,validation_data =Val, validation_steps= 9,steps_per_epoch=308,shuffle=True,verbose=1,epochs=300,callbacks=[reduce_lr,early_stop,checkpoint])
history = model.fit(Generator,validation_data =Val, validation_steps= 9,steps_per_epoch=308,shuffle=True,verbose=1,epochs=30)




Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Epoch 1/30


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/usr/local/lib/python3.10/site-packages/anyio/_core/_eventloop.py", line 74, in run

  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2303, in run

  File "/usr/local/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 216, in run

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete

  File "/usr/local/lib/python3.10/asyncio/base_events.py", 

In [ ]:
model.load_weights('G:/Segmentation _Multisensor/Weights/Binary_Building/DPP_Massa_Building.h5')
model.save('G:/Segmentation _Multisensor/Weights/Binary_Building/DPP_Massa_Building.h5')

In [ ]:
model = tf.keras.models.load_model('G:/Segmentation _Multisensor/Weights/Binary_Building/DPP_Massa_Building.h5')

In [ ]:
import os
gtappend = []
predappend =[]

addA = "G:/Segmentation _Multisensor/building/512/Test/I/x/"
addlabel = "G:/Segmentation _Multisensor/building/512/Test/G/y/"

for eg in os.listdir(addA):
    imageA = addA + str(eg)
    GT = addlabel + str(eg)
    x1 = (plt.imread(imageA))*255
    x2 = (plt.imread(GT))*255
    a = np.reshape(x1,(1,512,512,3))
    c = rgb_to_onehot(x2)
    y = model.predict(a)
    y = np.reshape(y,(512,512))
    result = np.where(y>0.5,1,0)
    predappend.append(result)
    gtappend.append(c)

g = np.stack(gtappend, axis=0)
p = np.stack(predappend, axis=0)

from sklearn.metrics import *
gt=(g.ravel()).astype('int')
pd=(p.ravel()).astype('int')
f1 = f1_score(gt,pd,average='macro')
kappa = cohen_kappa_score(gt,pd)
accuracy = accuracy_score(gt,pd)
jaccard = jaccard_score(gt,pd,average='macro')
precision = precision_score(gt,pd,average='macro')
recall = recall_score(gt,pd,average='macro')
print(np.unique(gt),np.unique(pd))
print("F1 SCORE:", f1)
print("Kappa:",kappa)
print("Accuracy:",accuracy)
print("Jaccard Score:",jaccard)
print("Precision:",precision)
print("Recall:",recall)

In [ ]:
def get_dice(y_true, y_pred):
    intersection = np.sum(y_true * y_pred, axis=(0, 1, 2))
    union = np.sum(y_true**2, axis=(0, 1, 2)) + np.sum(y_pred**2, axis=(0, 1, 2))
    dc = 2 * intersection / union
    return dc
 
dice_coeff = get_dice(g, p)
print("dc = ",np.mean(dice_coeff))

In [ ]:
import os
gtappend = []
predappend =[]
img = []
addA = "G:/Segmentation _Multisensor/building/512/Test/I/x/"
addlabel = "G:/Segmentation _Multisensor/building/512/Test/G/y/"

for eg in os.listdir(addA):
    imageA = addA + str(eg)
    GT = addlabel + str(eg)
    x1 = (plt.imread(imageA))*255
    x2 = (plt.imread(GT))*255
    a = np.reshape(x1,(1,512,512,3))
    c = rgb_to_onehot(x2)
    y = model.predict(a)
    y = np.reshape(y,(512,512))
    result = np.where(y>0.5,1,0)
    predappend.append(result)
    gtappend.append(c)
    img.append(x1)
    
g = np.stack(gtappend, axis=0)
p = np.stack(predappend, axis=0)
img1 = np.stack(img, axis=0)

In [ ]:
g.max()

In [ ]:
p.max()

In [ ]:
img1.max()

In [ ]:
import random
import cv2
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from skimage import measure
from skimage import filters

def Opening(image, filter_size):
    #Opening is just another name of erosion followed by dilation
    img=np.copy(image)
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(filter_size,filter_size))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return opening

def Closing(image,k):# closing is useful to detect the overall contour of a figure and opening is suitable to detect subpatterns. 
    kernel = np.ones((k, k), np.uint8)
    img=np.copy(image)
    img_close = cv2.morphologyEx(img, op= cv2.MORPH_CLOSE,kernel=kernel)
    return img_close

def Denoise(image,k):
    img=np.copy(image)
    struct=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(k,k))
    img=cv2.morphologyEx(img,cv2.MORPH_OPEN,struct)
    return img

def Binary(image, threshold, max_value = 1):
    img=np.copy(image)
    (t,masklayer)=cv2.threshold(img,threshold,max_value,cv2.THRESH_BINARY)
    return masklayer

def Gaussian_filter(image, sigma =1):
    img=np.copy(image)
    blur = filters.gaussian(img, sigma=sigma)
    return blur

def Find_threshold_otsu(image):
    t = filters.threshold_otsu(image)
    return t

def ExtractObjects(image):
    img=np.copy(image)
    blob_labels=measure.label(img,background=0)
    number_of_objects=np.unique(blob_labels)
    return blob_labels,number_of_objects

def post_processing(img):
    
    blur = Gaussian_filter(img, sigma=1)
    t = Find_threshold_otsu(blur)
    binary_img = Binary(blur,t)
    opened_img  = Opening(binary_img, filter_size = 3)
    blob_labels,number_of_objects = ExtractObjects(opened_img)
    
    return opened_img, number_of_objects, blob_labels 

In [ ]:
number = 9
pred = p[number,:,:]
opened_img, number_of_objects, blob_labels = post_processing(pred)
print("Number of Buildings in prediction: {}".format(number_of_objects[-1]))

gt = g[number,:,:]
opened_img1, number_of_objects1, blob_labels1 = post_processing(gt)
print("Number of Buildings in Groundtruth: {}".format(number_of_objects1[-1]))

f, axs = plt.subplots(1,5, figsize=(23,23))
img = img1[number,:,:,:]
img = np.reshape(img,(img.shape[0],img.shape[1],3))
# img = img.astype('uint8')
axs[0].imshow(img/255.0)
axs[0].set_title("Original")

gt = g[number,:,:]
gt = np.reshape(gt,(gt.shape[0],gt.shape[1]))
axs[1].imshow(gt)
axs[1].set_title("Ground Truth")

axs[2].imshow(opened_img)
axs[2].set_title("Prediction")


axs[3].imshow(blob_labels, cmap='gist_ncar')
axs[3].set_title("Number of buildings: {}".format(number_of_objects[-1]))

axs[4].imshow(blob_labels1, cmap='gist_ncar')
axs[4].set_title("Number of buildings in gt: {}".format(number_of_objects1[-1]))

f.show()